In [1]:
import phrase_entity_extraction as ph

In [2]:
st = "8645 D GOLD PEAK PL".lower()
max_address = ph.return_max_address3(ph.seq, st)
if st.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(st))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(st), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(st.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  D 

WORD ARRAY:  ['8645', 'd', 'gold', 'peak', 'pl'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['LETTER'], ['ALPHA'], ['ALPHA', 'GFEATURE'], ['WAY']]


8645 D GOLD PEAK PL,"[['DIGIT', 'ADR_HEAD'], ['LETTER'], ['ALPHA'], ['ALPHA', 'GFEATURE'], ['WAY']]"


In [3]:
ph.return_best_fit(ph.seq, st)

[[1, 2, 1, ['SUITE'], 'd']]

In [4]:
arr_cands = ph.get_markers(ph.seq, st, ['ADDRESS', 'POBOX', 'SUITE'])
arr_cands

[[0, 1, 1, ['SUITE'], '8645'], [1, 2, 1, ['SUITE'], 'd']]

In [5]:
import re
stst = 'st'

re.findall(r'^(st|ste)[a-z]+$', stst)

[]

In [6]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w("st 100"))).get_next_values()

[]

In [7]:
import re
rex = r'^[nsew]\d+(rd|st|th)'
re.findall(rex, 'n83st')

['st']